In [25]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append("../")

import yaml
import torch
from transformers import AutoTokenizer
from pytorch_lightning import Trainer
from src.model import chained_seq2seq
from src.data_modules_2 import STTDataModule2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
tokenizer_encoder = AutoTokenizer.from_pretrained("../model/opus-mt-nl-en")
ckpt_path = "../model/checkpoints/chained_seq2seq_logs/version_1/checkpoints/epoch=4-step=9075.ckpt"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [6]:
dm = STTDataModule2(num_workers = 12)
dm.setup("fit")
dm.setup("test")

train_dataloader = dm.train_dataloader()
val_dataloader = dm.val_dataloader()
test_dataloader = dm.test_dataloader()

In [36]:
model = chained_seq2seq()
model.eval()
model.to(device)
print()

In [11]:
trainer = Trainer(
    max_epochs=5,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,
    default_root_dir="../model/checkpoints/"
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
trainer.test(model, datamodule=dm, ckpt_path=ckpt_path)

Restoring states from the checkpoint path at ../model/checkpoints/chained_seq2seq_logs/version_1/checkpoints/epoch=4-step=9075.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ../model/checkpoints/chained_seq2seq_logs/version_1/checkpoints/epoch=4-step=9075.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/cer            1.9862076044082642
        test/loss             8.5058012008667
        test/wer            1.9862076044082642
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/loss': 8.5058012008667,
  'test/wer': 1.9862076044082642,
  'test/cer': 1.9862076044082642}]

In [27]:
with open("../model/checkpoints/chained_seq2seq_logs/version_1/hparams.yaml") as f:
    hparams = yaml.safe_load(f)
inferencer = chained_seq2seq.load_from_checkpoint(checkpoint_path=ckpt_path).to(device)


In [28]:
samp_src, samp_tgt = next(iter(test_dataloader))
src_ids = tokenizer_encoder(samp_src, return_tensors="pt", padding=True).input_ids.to(device)
tgt_ids = tokenizer_encoder(samp_tgt, return_tensors="pt",padding=True).input_ids.to(device)

In [37]:
outputs = model(src_ids, tgt_ids)
loss, logits = outputs[:2]
generated = inferencer.generate(logits)

In [38]:
generated

['de abbneazen zijn de oorspronkelijke inwoners van Australiëstraalië de de de de de',
 'Mijn toetsenbord zit mijn mijn mijn mijn mijn mijn mijn mijn mijn mijn mijn mijn',
 'Ze had de bank beschadigd met haar skateboard Ze Ze Ze Ze Ze',
 'Deze top heeft niet in alle opzichten aan de verwachtingen voldaan De De De De De De De De De',
 'waar je jeij je onderhoud doen waar waar waar waar waar waar waar waar waar waar waar waar',
 'de commissaris,dankt voor uw zeer nauwkeurige antwoorden De De De De De De De De De De',
 'Na het lezen van vele rec had ze eindelijk haar oog laten vallen op een laptop met een toetsenyybord',
 'Ik maak daar mijn mijn tegen dat mijn probeert van te makengradstr Ik Ik Ik',
 'Wat betekent deze in in?? De De De De Wat Wat Wat Wat Wat Wat Wat Wat Wat Wat Wat Wat Wat Wat',
 'zijnvon zijn aan zijn zijn zijn zijn zijn zijn zijn zijn zijn zijn zijn zijn zijn zijn zijn',
 'Pro we de te te verminderen of proberen wij energie energieverbruik te te dringen De De De De De',

In [33]:
samp_tgt

['de aboriginals zijn de oorspronkelijke bewoners van australië',
 'mijn toetsenbord zit vol stof',
 'ze had de bank beschadigd met haar skateboard',
 'deze top heeft niet in alle opzichten aan de verwachtingen voldaan',
 'waar laat jij je onderhoud doen',
 'mijnheer de commissaris bedankt voor uw bijzonder nauwkeurige antwoorden',
 'na het lezen van vele beoordelingen had ze eindelijk haar oog laten vallen op een laptop met een qwerty toetsenbord',
 'ik maak er ernstig bezwaar tegen wanneer men een volk tracht te demoniseren',
 'wat houdt dit vraagstuk in',
 'de tampons zijn op',
 'proberen wij kooldioxideemissies te beperken of proberen wij het energiegebruik terug te dringen',
 'marijke kent olivier nu al meer dan twee jaar',
 'mijn partij vindt dat onverkwikkelijk',
 'het voeren van brood aan eenden is eigenlijk ongezond voor de beesten',
 'juni dit jaar is veel te laat',
 'parket moet je stofzuigen tegels moet je dweilen']